# Overwrite Content with CSV

The script utilizes <code>pandas</code> dataframes to read in [311 data from Socrata]("https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9").  Export the pandas dataframe into CSV
file, overwrite service in ArcGIS Online.

----

**Directions:** Begin by importing the standard libraries we have been working with: <code>pandas</code>, <code>getpass</code>, and <code>arcgis</code>.
* <code>pandas</code> is a common toolkit for data science. It contains <code>dataframes</code> which are flexible data structures used for data manipulation and analysis.
* <code>getpass</code> for obtaining a passcode securely.
* <code>arcgis</code> for importing gis portal libraries.

In [2]:
# Import libraries
import pandas as pd
import getpass
from arcgis.gis import GIS
from arcgis import features

**Directions:** Query the JSON data layer using <code>pd.read_json()</code>. Establish the CSV output location.

311 provides both WGS and State Plane coordinates. We will drop the set of WGS coordinate fields (i.e., longitude and latitude) from the JSON data layer, and import the State Plane coordinates using field names that ArcGIS Online can automatically read and use to geocode.

**Note:** ArcGIS will automatically identify coordinates if they are formated as:
* X, Y
* Latitude, Longitude 

In [31]:
# Query Socrata directly
# df = pd.read_json("https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-03-24T00:00:00%27%20and%20%272019-03-25T23:59:59%27")
df = pd.read_json("https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$limit=25")
# Establish csv output
csv_output = r'D:\Scratch\NYC_311.csv'

#Show Data Frame
df.head()
df = df.drop('location', 1)
df = df.drop('latitude', 1)
df = df.drop('longitude', 1)
df.rename(columns={'x_coordinate_state_plane': 'X'}, inplace=True)
df.rename(columns={'y_coordinate_state_plane': 'Y'}, inplace=True)
df.columns

Index(['address_type', 'agency', 'agency_name', 'bbl', 'borough', 'city',
       'closed_date', 'community_board', 'complaint_type', 'created_date',
       'cross_street_1', 'cross_street_2', 'descriptor', 'due_date',
       'facility_type', 'incident_address', 'incident_zip',
       'intersection_street_1', 'intersection_street_2', 'location_type',
       'open_data_channel_type', 'park_borough', 'park_facility_name',
       'resolution_action_updated_date', 'resolution_description', 'status',
       'street_name', 'unique_key', 'X', 'Y'],
      dtype='object')

**Directions:** Export the dataframe to a CSV using the previously designated output location, then view a preview of this CSV.

**Note:** The CSV should have 25 records based on the <code>$limit=25"</code> segment in the Socrata query. Using a small number of records to test the code is preferable. Once you have ensured the code works, you can [get more complicated with your queries]("https://dev.socrata.com/foundry/data.cityofnewyork.us/fhrw-4uyv"). (**Hint:** The API documentation for each dataset shows how to use <code>pandas</code> to query the endpoints.) 

In [32]:
df.to_csv(csv_output, sep=',', index=False)

In [33]:
# read the initial csv
csv1 = csv_output
cities_df_1 = pd.read_csv(csv1)
cities_df_1

,address_type,agency,agency_name,bbl,borough,city,closed_date,community_board,complaint_type,created_date,...,open_data_channel_type,park_borough,park_facility_name,resolution_action_updated_date,resolution_description,status,street_name,unique_key,X,Y
0,ADDRESS,NYPD,New York City Police Department,2.024550e+09,BRONX,BRONX,NaN,04 BRONX,Noise - Residential,2019-03-24T02:14:56.000,...,PHONE,BRONX,Unspecified,NaN,Your complaint has been forwarded to the New Y...,Open,EAST 164 STREET,42034978,1006603,241526
1,ADDRESS,NYPD,New York City Police Department,2.028390e+09,BRONX,BRONX,NaN,04 BRONX,Noise - Residential,2019-03-24T02:14:27.000,...,PHONE,BRONX,Unspecified,NaN,Your complaint has been forwarded to the New Y...,Open,EAST 169 STREET,42035194,1007141,244209
2,ADDRESS,NYPD,New York City Police Department,1.017180e+09,MANHATTAN,NEW YORK,NaN,10 MANHATTAN,Noise - Residential,2019-03-24T02:13:25.000,...,PHONE,MANHATTAN,Unspecified,NaN,Your complaint has been forwarded to the New Y...,Open,WEST 119 STREET,42037232,998679,232128
3,INTERSECTION,NYPD,New York City Police Department,NaN,QUEENS,JAMAICA,NaN,12 QUEENS,Noise - Vehicle,2019-03-24T02:12:34.000,...,PHONE,QUEENS,Unspecified,2019-03-24T02:23:33.000,Your complaint has been received by the Police...,Assigned,NaN,42036949,1043340,193789
4,ADDRESS,NYPD,New York City Police Department,3.069198e+09,BROOKLYN,BROOKLYN,NaN,13 BROOKLYN,Blocked Driveway,2019-03-24T02:11:08.000,...,PHONE,BROOKLYN,Unspecified,NaN,Your complaint has been forwarded to the New Y...,Open,WEST 15 PLACE,42036852,988795,152821
5,ADDRESS,DOHMH,Department of Health and Mental Hygiene,1.020700e+09,MANHATTAN,NEW YORK,NaN,09 MANHATTAN,Smoking,2019-03-24T02:10:51.000,...,ONLINE,MANHATTAN,Unspecified,NaN,The Department of Health and Mental Hygiene wi...,Open,WEST 139 STREET,42032351,997751,238682
6,ADDRESS,NYPD,New York City Police Department,2.023810e+09,BRONX,BRONX,2019-03-24T02:25:33.000,03 BRONX,Noise - Residential,2019-03-24T02:09:51.000,...,PHONE,BRONX,Unspecified,2019-03-24T02:25:12.000,The Police Department responded to the complai...,Closed,EAST 160 STREET,42032187,1008590,239102
7,ADDRESS,NYPD,New York City Police Department,2.041940e+09,BRONX,BRONX,NaN,10 BRONX,Noise - Residential,2019-03-24T02:09:51.000,...,PHONE,BRONX,Unspecified,NaN,Your complaint has been forwarded to the New Y...,Open,HOBART AVENUE,42036088,1030891,248454
8,ADDRESS,NYPD,New York City Police Department,2.026908e+09,BRONX,BRONX,NaN,02 BRONX,Noise - Commercial,2019-03-24T02:08:53.000,...,PHONE,BRONX,Unspecified,2019-03-24T02:48:58.000,Your complaint has been received by the Police...,Assigned,WESTCHESTER AVENUE,42033921,1012198,238417
9,ADDRESS,NYPD,New York City Police Department,2.025110e+09,BRONX,BRONX,NaN,04 BRONX,Illegal Parking,2019-03-24T02:08:31.000,...,PHONE,BRONX,Unspecified,NaN,Your complaint has been forwarded to the New Y...,Open,WEST 163 STREET,42034137,1003804,242458


In [34]:
# print the number of records in this csv
cities_df_1.shape

(25, 30)

**Directions:** Connect to the GIS portal, add the CSV as an item, and then publish as a feature service.

**Note:** When publishing as a feature serivce, you MUST specify spatial reference, including the <code>latestWkid</code> and the <code>wkid</code>. 

In [27]:
# Connect to the GIS
# password = getpass()
gis = GIS("https://www.arcgis.com",'nycpython1')

Enter password: ········


In [28]:
# add the csv as an item
title = 'NYC_311_2019_SP'
item_prop = {'title': title}
csv_item = gis.content.add(item_properties=item_prop, data=csv1)
csv_item

<Item title:"NYC_311_2019_SP" type:CSV owner:alex.brown.localgovne>

In [30]:
# publish the csv item into a feature layer
new_item = csv_item.id
feature_layer_item = gis.content.get(new_item)

# pparms = {"name": '311_All_2019_SP', "targetSR": "wkid": 102718, "latestWkid": 2263},
#           "sourceSR": {"wkid": 102718, "latestWkid": 2263}}
        
pparms = {"name": 'NYC_311_2019_SP', "targetSR": {"latestWkid": 102718, "wkid":2263}}
print(type(pparms))
feature_layer_item.publish(publish_parameters=pparms)
# feature_layer_item.publish()

<class 'dict'>


Exception: Job failed.

**Directions:** After publishing, ensure the feature service exists in the GIS portal and add it to a web map.

In [49]:
# Search by title
search = gis.content.search(query="title: NYC_311_2019_SP", item_type="Feature Service")
search

[<Item title:"NYC_311_2019_SP" type:Feature Layer Collection owner:alex.brown.localgovne>]

In [50]:
# From search list, return the index for service one is interseted in connecting to
item = search[0]
itemID = item.id
itemID

'41987c6e0aae43cfa74278649ddf74bd'

In [51]:
# Establish connection to Feature Layer item
feature_layer_new = gis.content.get(itemID)


In [52]:
map1 = gis.map('New York, NY')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [53]:
map1.add_layer(feature_layer_new)